<a href="https://colab.research.google.com/github/okana2ki/intro-to-AI/blob/main/timetable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: google driveからファイルを読み込む

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import re

# ファイルのパスを指定します。
file_path = '/content/drive/MyDrive/Colab_files/2025keiei1.xlsx'  # 自分のファイルのパスに変更してください

# Excelファイルの読み込み（ヘッダーなし）
df = pd.read_excel(file_path, header=None)

# ヘッダー部の取得
title = df.iloc[0, 0]  # 1行目（タイトル）
weekdays = df.iloc[1, 3:3+5].tolist()  # 2行目、例としてD～H列を曜日ラベルと仮定
labels = df.iloc[2, 3:3+5].tolist()  # 3行目、項目ラベル（例）

# 定義：各時限・学期の行番号のブロック情報
# ※実際は0-indexedに直す必要がある（例：Excelの5行目はdf.iloc[4]）
blocks = {
    "1限": {"前期": (4, 9), "通年": (11, 15), "後期": (17, 22)},
    "2限": {"前期": (24, 30), "通年": (32, 37), "後期": (39, 46)},
    "3限": {"前期": (48, 53), "通年": (55, 62), "後期": (64, 69)},
    "4限": {"前期": (71, 76), "通年": (78, 82), "後期": (84, 89)},
    "5限": {"前期": (91, 96), "通年": (98, 102), "後期": (104, 109)}
}

# 曜日ごとの列マッピング（0-indexedに直す）
columns_map = {
    "月": {"年次": 3, "科目": 4, "コード": 7, "教員": 8, "教室": 9},
    "火": {"年次": 10, "科目": 11, "コード": 12, "教員": 13, "教室": 14},
    "水": {"年次": 15, "科目": 16, "コード": 19, "教員": 20, "教室": 21},
    "木": {"年次": 22, "科目": 23, "コード": 26, "教員": 27, "教室": 29},
    "金": {"年次": 30, "科目": 31, "コード": 34, "教員": 35, "教室": 36}
}

def normalize_subject(subject_str):
    """
    科目名に付与されるクラス情報の括弧類の表記揺れを正規化する。
    例：全角・半角の違い、前後のスペースの除去など
    """
    if pd.isna(subject_str):
        return ""
    # 全角スペースを半角に変換し、前後の空白を除去
    norm = subject_str.replace("　", " ").strip()
    # 括弧類の統一（例：〈,〉, <, >, ［,］, [, ] などを標準形式に）
    norm = re.sub(r"[〈<\[]", "〈", norm)
    norm = re.sub(r"[〉>\]]", "〉", norm)
    norm = re.sub(r"[（(]", "（", norm)
    norm = re.sub(r"[）)]", "）", norm)
    norm = re.sub(r"[［\[]", "［", norm)
    norm = re.sub(r"[］\]]", "］", norm)
    return norm

# データを格納する構造
timetable = {}

# 各時限・学期ごとにデータを抽出
for period, sem_dict in blocks.items():
    timetable[period] = {}
    for sem, (start, end) in sem_dict.items():
        # 各ブロック内の行を取得
        block_df = df.iloc[start:end]
        # 曜日ごとの情報を格納する辞書
        day_data = {}
        for day, cols in columns_map.items():
            entries = []
            for idx, row in block_df.iterrows():
                # 各項目の値を取得
                year = row.iloc[cols["年次"]]
                subject = normalize_subject(row.iloc[cols["科目"]])
                code = row.iloc[cols["コード"]]
                teacher = row.iloc[cols["教員"]]
                room = row.iloc[cols["教室"]]
                # 空でなければ情報をまとめる
                if pd.notna(subject) and subject != "":
                    entry = {
                        "年次": year,
                        "科目": subject,
                        "コード": code,
                        "教員": teacher,
                        "教室": room
                    }
                    entries.append(entry)
            day_data[day] = entries
        timetable[period][sem] = day_data

# 例外処理：木曜3限通年のセルの内容を無条件に上書き
if "3限" in timetable and "通年" in timetable["3限"]:
    timetable["3限"]["通年"]["木"] = [
        {"年次": "➊", "科目": "進路研究演習１ 〈a 武次玄三〉", "コード": 21, "教員": None, "教室": 5401},
        {"年次": "➊", "科目": "進路研究演習１ 〈b 安井　誠〉", "コード": 22, "教員": None, "教室": 5402},
        {"年次": "➊", "科目": "進路研究演習１ 〈c 雜賀智子〉", "コード": 23, "教員": None, "教室": 5403},
        {"年次": "➊", "科目": "進路研究演習１ 〈d 堀下　翔〉", "コード": 24, "教員": None, "教室": 5404},
        {"年次": "➊", "科目": "進路研究演習１ 〈e 岩切道雄〉", "コード": 25, "教員": None, "教室": 5405},
        {"年次": "➊", "科目": "進路研究演習１ 〈f 若松朋也〉", "コード": 30, "教員": None, "教室": 3104},
        {"年次": "➊", "科目": "進路研究演習１ 〈g 藏屋滉平〉", "コード": 31, "教員": None, "教室": 3203},
    ]

In [6]:
# HTMLテーブルの生成
html = []
html.append('<html>')
html.append('<head><meta charset="UTF-8"><style>')
html.append('table { border-collapse: collapse; }')
html.append('th, td { border: 1px solid #000; padding: 5px; }')
html.append('</style></head>')
html.append('<body>')

# タイトル行
html.append(f'<h2>{title}</h2>')

# 曜日ヘッダー（例として固定）
weekdays_list = ["月", "火", "水", "木", "金"]
html.append('<table>')
html.append('<thead>')
# ヘッダー：曜日行
html.append('<tr>')
html.append('<th>時限・学期</th>')
for day in weekdays_list:
    html.append(f'<th>{day}</th>')
html.append('</tr>')
html.append('</thead>')
html.append('<tbody>')

# 各時限、各学期のデータ出力
for period in ["1限", "2限", "3限", "4限", "5限"]:
    for sem in ["前期", "通年", "後期"]:
        html.append('<tr>')
        html.append(f'<td>{period} {sem}</td>')
        for day in weekdays_list:
            cell_entries = timetable.get(period, {}).get(sem, {}).get(day, [])
            if cell_entries:
                cell_html = ""
                for entry in cell_entries:
                    # 各エントリを改行で区切って表示
                    parts = []
                    if pd.notna(entry.get("年次")):
                        parts.append(f'年次: {entry.get("年次")}')
                    parts.append(f'科目: {entry.get("科目")}')
                    if pd.notna(entry.get("コード")):
                        parts.append(f'コード: {entry.get("コード")}')
                    if pd.notna(entry.get("教員")):
                        parts.append(f'教員: {entry.get("教員")}')
                    if pd.notna(entry.get("教室")):
                        parts.append(f'教室: {entry.get("教室")}')
                    cell_html += "<div>" + " | ".join(parts) + "</div>"
                html.append(f'<td>{cell_html}</td>')
            else:
                html.append('<td></td>')
        html.append('</tr>')

html.append('</tbody>')
html.append('</table>')
html.append('</body></html>')

html_output = "\n".join(html)

# HTMLファイルに書き込む
with open('/content/drive/MyDrive/Colab_files/timetable.html', 'w', encoding='utf-8') as file:
    file.write(html_output)